In [19]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd
import numpy as np

from src.features import build_features

import glob

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]
COLUMNAS_POBLACION_FONASA = ["2018", "2019", "2020", "2021", "2022"]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

COMUNAS_SSMC = ["Santiago", "Maipú", "Estación Central", "Cerrillos"]
COMUNAS_SSMS = [
    "San Bernardo",
    "Pedro Aguirre Cerda",
    "La Cisterna",
    "San Joaquín",
    "Lo Espejo",
    "San Miguel",
    "Paine",
    "Calera de Tango",
    "Buin",
    "El Bosque",
]



## Obtener poblacion INE y FONASA


In [21]:
df_ine = pd.read_csv("../data/processed/df_ine.csv")
df_fonasa = pd.read_csv("../data/processed/df_fonasa.csv", dtype={"ANO_INFORMACION": str})

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_41180/1139144624.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fonasa = pd.read_csv("../data/processed/df_fonasa.csv", dtype={"ANO_INFORMACION": str})


## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO
4. SSMC
5. SSMS


### INE


In [22]:
ESTRATOS_A_CALCULAR_INE = {}

# Obtiene estratos a calcular
df_pais = df_ine.copy()
ESTRATOS_A_CALCULAR_INE["Pais"] = df_pais

for region in sorted(df_pais["M"].unique()):
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR_INE[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR_INE["SSMO"] = df_ssmo

df_ssmc = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMC)]
ESTRATOS_A_CALCULAR_INE["SSMC"] = df_ssmc

df_ssms = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMS)]
ESTRATOS_A_CALCULAR_INE["SSMS"] = df_ssms

### FONASA


In [23]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in sorted(df_pais_fonasa["REGION"].fillna("").unique()):
    if region:
        df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
        ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO ORIENTE'")
df_ssmc_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO CENTRAL'")
df_ssms_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO SUR'")

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMS"] = df_ssms_fonasa

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.


### INE


In [24]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "",  # Todo el pais
    "hombres": "hombre_mujer == 1",  # Hombres
    "mujeres": "hombre_mujer == 2",  # Mujeres
    "recien_nacidos_vivos": "Edad == 0",  # Recien Nacidos Vivos
    "sobre_15": "Edad >= 15",  # Sobre 15 anios
    "sobre_20": "Edad >= 20",  # Sobre 20 anios
    "sobre_45": "Edad >= 45",  # Sobre 45 anios
    "sobre_60": "Edad >= 60",  # Sobre 60 anios
    "sobre_65": "Edad >= 65",  # Sobre 65 anios
    "sobre_75": "Edad >= 75",  # Sobre 75 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24",  # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44",  # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59",  # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70",  # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74",  # Entre 60 a 74 anios
}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.


In [25]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados_ine = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_INE, query_strings, COLUMNAS_POBLACION_INE
)
# Une todos los resultados en un unico DataFrame
poblaciones_estratos_calculados_ine = pd.concat(poblaciones_estratos_calculados_ine)
poblaciones_estratos_calculados_ine = poblaciones_estratos_calculados_ine.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

### FONASA


In [26]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "",  # Todo el pais
    "hombres": "SEXO == 'HOMBRE'",  # Hombres
    "mujeres": "SEXO == 'MUJER'",  # Mujeres
    "recien_nacidos_vivos": "",  # Recien Nacidos Vivos
    "sobre_15": "EDAD_TRAMO >= 15",  # Sobre 15 anios
    "sobre_20": "EDAD_TRAMO >= 20",  # Sobre 20 anios
    "sobre_45": "EDAD_TRAMO >= 45",  # Sobre 45 anios
    "sobre_60": "EDAD_TRAMO >= 60",  # Sobre 60 anios
    "sobre_65": "EDAD_TRAMO >= 65",  # Sobre 65 anios
    "sobre_75": "EDAD_TRAMO >= 75",  # Sobre 75 anios
    "entre_15_a_24_anios": "EDAD_TRAMO >= 15 and EDAD_TRAMO < 25",  # Entre 15 y 24 anios
    "entre_25_a_44_anios": "EDAD_TRAMO >= 25 and EDAD_TRAMO < 45",  # Entre 25 y 44 anios
    "entre_45_a_59_anios": "EDAD_TRAMO >= 45 and EDAD_TRAMO < 60",  # Entre 45 y 59 anios
    "entre_60_y_70_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO <= 70",  # Entre 60 y 70 anios
    "entre_60_a_74_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO < 75",  # Entre 60 a 74 anios
    "mayores_a_75_anios": "EDAD_TRAMO >= 75",  # Mayores a 75 anios
}

In [27]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados_fonasa = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_FONASA, query_strings, "CUENTA_BENEFICIARIOS", "FONASA"
)

# Une todos los resultaods en un unico DataFrame
poblaciones_estratos_calculados_fonasa = pd.concat(poblaciones_estratos_calculados_fonasa)
poblaciones_estratos_calculados_fonasa = poblaciones_estratos_calculados_fonasa.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

## Obtener porcentaje de población FONASA total

En este caso, se quiere saber el % de población FONASA a nivel Nacional. Esto se utilizará para obtener los casos FONASA al 2035.


In [28]:
# Obtiene poblacion INE entre 2018 y 2022
poblacion_INE_2018_y_2022 = poblaciones_estratos_calculados_ine.query(
    "`Edad Incidencia` == 'todos'"
).set_index("Estrato")[COLUMNAS_POBLACION_FONASA]

# Calcula la suma de poblacion de todo el periodo
poblacion_INE_2018_y_2022["suma_poblacion"] = poblacion_INE_2018_y_2022.sum(axis=1)

# Obtiene poblacion FONASA entre 2018 y 2022
poblacion_FONASA_2018_y_2022 = poblaciones_estratos_calculados_fonasa.query(
    "`Edad Incidencia` == 'todos'"
).set_index("Estrato")[COLUMNAS_POBLACION_FONASA]

# Calcula la suma de poblacion de todo el periodo
poblacion_FONASA_2018_y_2022["suma_poblacion"] = poblacion_FONASA_2018_y_2022.sum(axis=1)

# Obtiene porcentaje de FONASA por anio y promedio entre 2018 - 2022
porcentaje_fonasa = poblacion_FONASA_2018_y_2022 / poblacion_INE_2018_y_2022
porcentaje_fonasa = porcentaje_fonasa.rename(columns={"suma_poblacion": "porcentaje_fonasa_acumulado"})

In [29]:
# Crea un resumen del porcentaje FONASA
resumen_porcentaje_fonasa = pd.concat([poblacion_INE_2018_y_2022, poblacion_FONASA_2018_y_2022, porcentaje_fonasa], axis=1)

In [30]:
print("Los porcentajes de FONASA son:")
display(round(resumen_porcentaje_fonasa * 100, 1))

Los porcentajes de FONASA son:


,2018,2019,2020,2021,2022,suma_poblacion,2018,2019,2020,2021,2022,suma_poblacion,2018,2019,2020,2021,2022,porcentaje_fonasa_acumulado
Estrato,,,,,,,,,,,,,,,,,,
Pais,1875140500,1910721600,1945831000,1967836300,1982856300,9682385700,1389961300,1462876000,1491435900,1503862600,1544786400,7392922200,74.1,76.6,76.6,76.4,77.9,76.4
Antofagasta,64502200,66856300,69185400,70374600,70963700,341882200,41373100,44261300,45794400,46958400,49532900,227920100,64.1,66.2,66.2,66.7,69.8,66.7
Arica y Parinacota,24190100,24703600,25211000,25538000,25772200,125414900,18927000,20089000,20578700,21277400,22372000,103244100,78.2,81.3,81.6,83.3,86.8,82.3
Atacama,30783500,31130700,31470900,31673700,31800400,156859200,24093500,25207100,25737700,25992800,26833500,127864600,78.3,81.0,81.8,82.1,84.4,81.5
Aysén del General Carlos Ibáñez del Campo,10602300,10668000,10729700,10773700,10804700,53578400,7561700,7950100,8112500,8134500,8241500,40000300,71.3,74.5,75.6,75.5,76.3,74.7
Biobío,164546000,165474400,166369600,167059000,167626900,831075900,133156100,137390000,139116600,139548400,141999600,691210700,80.9,83.0,83.6,83.5,84.7,83.2
Coquimbo,80721300,82172600,83609600,84807900,85876900,417188300,64813700,68324800,69993500,70778100,72596000,346506100,80.3,83.1,83.7,83.5,84.5,83.1
La Araucanía,100142000,100796500,101434300,101954800,102402900,506730500,84477100,87938900,89945300,90873700,93065100,446300100,84.4,87.2,88.7,89.1,90.9,88.1
Libertador General Bernardo O'Higgins,96648600,97886800,99106300,100095900,100955200,494692800,77352400,80665000,82619600,83026000,85762000,409425000,80.0,82.4,83.4,82.9,85.0,82.8


En este apartado se leerán las incidencias, junto a las poblaciones respectivas donde se tienen que aplicar tales incidencias. Posteriormente, se calcularán los casos teóricos para cada problema de salud, y para cada estrato definido.


## Lectura de Incidencias por problema de salud


In [121]:
RUTA_PLANILLA_INCIDENCIAS = "../data/external/incidencias_y_prevalencias_INT.xlsx"
COLUMNAS_A_UTILIZAR = [
    "Diagnostico",
    "Diagnosticos Contenidos",
    "Estadística",
    "Casos (Cada 100.000)",
    "Edad Incidencia",
    "Área de Influencia",
    "Porcentaje Hospitalizados de Ambulatorios",
]


# Lee planilla de incidencias
incidencias = pd.read_excel(RUTA_PLANILLA_INCIDENCIAS, usecols=COLUMNAS_A_UTILIZAR)

# Filtra los diagnosticos limitados por oferta
limitados_por_oferta = incidencias.query("`Estadística` == 'Acotado por oferta'")

# Convierte las incidencia a numeros e imputa las faltantes como NaN
incidencias["Casos (Cada 100.000)"] = pd.to_numeric(
    incidencias["Casos (Cada 100.000)"], errors="coerce"
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Casos (Cada 100.000)"] / 100000

# Corrige prevalencias en 5 anios
idx_prevalencias = incidencias["Estadística"] == "Prevalencia"
incidencias.loc[idx_prevalencias, "rate_incidencia"] = (
    incidencias.loc[idx_prevalencias, "rate_incidencia"] / 5
)

## Estimacion de casos con INE


In [122]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias_ine = incidencias.merge(
    poblaciones_estratos_calculados_ine, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias_ine[COLUMNAS_POBLACION_INE] = union_poblaciones_e_incidencias_ine[
    COLUMNAS_POBLACION_INE
].multiply(union_poblaciones_e_incidencias_ine["rate_incidencia"], axis="index")

## Estimacion de casos con FONASA


In [123]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias_fonasa = incidencias.merge(
    poblaciones_estratos_calculados_fonasa, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias_fonasa[COLUMNAS_POBLACION_FONASA] = (
    union_poblaciones_e_incidencias_fonasa[COLUMNAS_POBLACION_FONASA].multiply(
        union_poblaciones_e_incidencias_fonasa["rate_incidencia"], axis="index"
    )
)

## Estimacion de casos FONASA, utilizando la poblacion INE y el % historico de la poblacion FONASA


In [124]:
# Copia los casos INE ya calculados
union_poblaciones_e_incidencias_fonasa_desde_ine = union_poblaciones_e_incidencias_ine.copy()

# Agrega el porcentaje FONASA de cada uno de los estratos calculados
union_poblaciones_e_incidencias_fonasa_desde_ine = (
    union_poblaciones_e_incidencias_fonasa_desde_ine.merge(
        porcentaje_fonasa["porcentaje_fonasa_acumulado"], how="left", left_on="Estrato", right_index=True
    )
)

# # Multiplica el % de FONASA con la poblacion INE
union_poblaciones_e_incidencias_fonasa_desde_ine[COLUMNAS_POBLACION_INE] = (
    union_poblaciones_e_incidencias_fonasa_desde_ine[COLUMNAS_POBLACION_INE].mul(
        union_poblaciones_e_incidencias_fonasa_desde_ine["porcentaje_fonasa_acumulado"], axis=0
    )
)

## Acotación de casos según área de influencia para el INT

Una vez obtenido los casos proyectados hasta el 2035, se procederá a acotar cada uno de los diagnósticos y sus casos según su área de influencia determinada.


In [125]:
area_de_infl_INT = union_poblaciones_e_incidencias_fonasa_desde_ine.copy()

# Separa las areas de influencia por comas, para dejarlas en una lista
area_de_infl_INT["Área de Influencia"] = area_de_infl_INT["Área de Influencia"].str.split(",")
area_de_infl_INT = area_de_infl_INT.explode("Área de Influencia")
area_de_infl_INT["Área de Influencia"] = area_de_infl_INT["Área de Influencia"].str.strip()

# Solamente deja las filas donde el Area de Influencia coincida con el Estrato
area_de_infl_INT = (
    area_de_infl_INT.query("`Área de Influencia` == `Estrato`")
    .groupby("Diagnostico")[COLUMNAS_POBLACION_INE]
    .sum()
)
# Une el resultado de casos por area de influencia a las incidencias iniciales
area_de_infl_INT = (
    incidencias.set_index("Diagnostico")
    .merge(area_de_infl_INT, how="left", left_index=True, right_index=True)
    .reset_index()
)

# Elimina los diagnosticos acotados por oferta calculados incorrectamente
area_de_infl_INT = area_de_infl_INT.query("`Estadística` != 'Acotado por oferta'")

In [126]:
# Obtiene los casos de los diags acotado por oferta
area_de_infl_INT_acotados_por_oferta = limitados_por_oferta.copy()
for anio_ine in COLUMNAS_POBLACION_INE:
    area_de_infl_INT_acotados_por_oferta[anio_ine] = area_de_infl_INT_acotados_por_oferta[
        "Casos (Cada 100.000)"
    ]

In [127]:
# Une los casos acotados por oferta a los casos por incidencia y prevalencia
area_de_infl_INT = pd.concat([area_de_infl_INT, area_de_infl_INT_acotados_por_oferta])

In [128]:
dfs_a_guardar = {
    "poblacion_INE": poblaciones_estratos_calculados_ine,
    "poblacion_FONASA": poblaciones_estratos_calculados_fonasa,
    "porcentaje_FONASA_por_estrato": resumen_porcentaje_fonasa.reset_index(),
    "casos_teoricos_INE": union_poblaciones_e_incidencias_ine,
    "casos_teoricos_FONASA_desde_INE": union_poblaciones_e_incidencias_fonasa_desde_ine,
    "casos_area_de_influencia_INT": area_de_infl_INT,
}

with pd.ExcelWriter("../data/interim/casos_teoricos_diagnosticos.xlsx") as writer:
    for sheet_name, df_a_guardar in dfs_a_guardar.items():
        try:
            df_a_guardar = df_a_guardar.drop(columns="rate_incidencia")
        except KeyError:
            pass

        df_a_guardar.to_excel(writer, sheet_name=sheet_name, index=False)